In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms

In [22]:
from matplotlib.pyplot import imshow
%matplotlib inline

In [23]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [24]:
trans = transforms.Compose([
    transforms.Resize((64,128))
])

train_data = torchvision.datasets.ImageFolder(root='Origin_data/', transform=trans)

In [25]:
for num, value in enumerate(train_data):
    data, label = value
    #print(num, data, label)
    if(label == 0):
        data.save('custom_data/train_data/JUNI/%d_%d.jpeg'%(num, label))
    else:
        data.save('custom_data/train_data/Wonbin/%d_%d.jpeg'%(num, label))
    

In [26]:
#이미지 사이즈 조정 완료

In [27]:
trans = transforms.Compose([
    transforms.ToTensor()
])

train_data = torchvision.datasets.ImageFolder(root='./custom_data/train_data/', transform=trans)

In [28]:
data_loader = DataLoader(dataset = train_data, batch_size = 8, shuffle = True, num_workers=2)

In [29]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,6,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6,16,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer3 = nn.Sequential(
            nn.Linear(16*13*29, 120),
            nn.ReLU(),
            nn.Linear(120,2)
        )
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.shape[0], -1)
        out = self.layer3(out)
        return out

In [30]:
net = CNN().to(device)
test_input = (torch.Tensor(3,3,64,128)).to(device)
test_out = net(test_input)

In [31]:
optimizer = optim.Adam(net.parameters(), lr=0.005)
loss_func = nn.CrossEntropyLoss().to(device)

In [32]:
total_batch = len(data_loader)

epochs = 5
for epoch in range(epochs):
    avg_cost = 0.0
    for num, data in enumerate(data_loader):
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        out = net(imgs)
        loss = loss_func(out, labels)
        loss.backward()
        optimizer.step()
        
        avg_cost += loss / total_batch
        
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
print('Learning Finished!')

[Epoch:1] cost = 0.6956681609153748
[Epoch:2] cost = 0.6934946179389954
[Epoch:3] cost = 0.6859956383705139
[Epoch:4] cost = 0.654465913772583
[Epoch:5] cost = 0.6607041954994202
Learning Finished!


In [33]:
torch.save(net.state_dict(), "./model/model.pth")

In [34]:
new_net = CNN().to(device)

In [35]:
new_net.load_state_dict(torch.load('./model/model.pth'))

<All keys matched successfully>

In [36]:
print(net.layer1[0])
print(new_net.layer1[0])

print(net.layer1[0].weight[0][0][0])
print(new_net.layer1[0].weight[0][0][0])

net.layer1[0].weight[0] == new_net.layer1[0].weight[0]

Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
tensor([-0.1296, -0.0349, -0.0552, -0.0592,  0.0552], grad_fn=<SelectBackward>)
tensor([-0.1296, -0.0349, -0.0552, -0.0592,  0.0552], grad_fn=<SelectBackward>)


tensor([[[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]],

        [[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]],

        [[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]]])

In [37]:
trans=torchvision.transforms.Compose([
    transforms.Resize((64,128)),
    transforms.ToTensor()
])
test_data = torchvision.datasets.ImageFolder(root='./custom_data/test_data/', transform=trans)

In [38]:
for num, value in enumerate(test_data):
    data, label = value
    print(num, data, label)

0 tensor([[[0.2000, 0.2039, 0.2039,  ..., 0.2549, 0.2588, 0.2627],
         [0.2000, 0.2078, 0.2039,  ..., 0.2549, 0.2549, 0.2667],
         [0.1922, 0.2000, 0.2000,  ..., 0.2588, 0.2627, 0.2706],
         ...,
         [0.4706, 0.5333, 0.5961,  ..., 0.6392, 0.6392, 0.6431],
         [0.4627, 0.5020, 0.5294,  ..., 0.6431, 0.6314, 0.6275],
         [0.4314, 0.4588, 0.4667,  ..., 0.6118, 0.6118, 0.6118]],

        [[0.2039, 0.2039, 0.2039,  ..., 0.2784, 0.2824, 0.2863],
         [0.2039, 0.2078, 0.2039,  ..., 0.2706, 0.2706, 0.2824],
         [0.1961, 0.2000, 0.2000,  ..., 0.2627, 0.2667, 0.2745],
         ...,
         [0.4784, 0.5412, 0.6000,  ..., 0.6627, 0.6627, 0.6667],
         [0.4627, 0.5020, 0.5333,  ..., 0.6510, 0.6392, 0.6353],
         [0.4314, 0.4588, 0.4667,  ..., 0.6196, 0.6196, 0.6196]],

        [[0.1961, 0.1961, 0.1961,  ..., 0.2784, 0.2824, 0.2863],
         [0.1922, 0.2000, 0.1961,  ..., 0.2745, 0.2745, 0.2863],
         [0.1843, 0.1922, 0.1922,  ..., 0.2706, 0.2745, 

In [39]:
test_set = DataLoader(dataset = test_data, batch_size = len(test_data))

In [40]:
with torch.no_grad():
    for num, data in enumerate(test_set):
        imgs, label = data
        imgs = imgs.to(device)
        label = label.to(device)
        
        prediction = net(imgs)
        
        correct_prediction = torch.argmax(prediction, 1) == label
        
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())

Accuracy: 0.6000000238418579
